### Importing packages

In [1]:
import pandas as pd
import numpy as np
from shapely.wkt import loads as load_wkt
import geopandas as gpd

In [2]:
zipcodes = pd.read_csv("../data/ZIPCODES.csv")

In [3]:
def generate_location_area_dim(zipcodes):
    """
    Generate a location area dimension DataFrame from the given zipcodes DataFrame.

    Args:
        zipcodes (DataFrame): A DataFrame containing ZIP codes and their geometries.

    Returns:
        DataFrame: A DataFrame containing the location area dimension data.
    """
    # Convert 'the_geom' to geometry and calculate centroids
    zipcodes['geometry'] = zipcodes['the_geom'].apply(load_wkt)
    gdf = gpd.GeoDataFrame(zipcodes, geometry='geometry')
    gdf['centroid'] = gdf['geometry'].centroid
    gdf['centroid_latitude'] = gdf['centroid'].y
    gdf['centroid_longitude'] = gdf['centroid'].x

    # Generate LocationAreaKey by concatenating latitude and longitude without special characters
    gdf['LocationAreaKey'] = (
        gdf['centroid_longitude'].astype(str).str.replace('.', '', regex=False).str.replace('-', '', regex=False).str[:8] +
        gdf['centroid_latitude'].astype(str).str.replace('.', '', regex=False).str.replace('-', '', regex=False).str[:8]
    )

    # Prepare the location area dimension DataFrame
    location_area_dim = pd.DataFrame({
        'LocationAreaKey': gdf['LocationAreaKey'],
        'Zipcode': gdf['ZIPCODE'],
        'MailCity': gdf['MAIL_CITY'],
        'ShapeLength': gdf['Shape_Leng'],
        'ShapeArea': gdf['Shape_Area'],
        'CentroidLatitude': gdf['centroid_latitude'],
        'CentroidLongitude': gdf['centroid_longitude']
    })

    return location_area_dim


In [4]:
test = generate_location_area_dim(zipcodes)

In [5]:
test

,LocationAreaKey,Zipcode,MailCity,ShapeLength,ShapeArea,CentroidLatitude,CentroidLongitude
0,7714293738966626,20812,GLEN ECHO,14504.506513,9.810937e+06,38.966627,-77.142937
1,7711806238956479,20816,BETHESDA,58716.124845,1.175655e+08,38.956480,-77.118062
2,7716110538973139,20818,CABIN JOHN,31103.235156,3.599683e+07,38.973140,-77.161105
3,7709644338994223,20894,BETHESDA,1298.401522,2.547457e+04,38.994224,-77.096444
4,7709647338995261,20892,BETHESDA,140.742936,7.448763e+02,38.995261,-77.096473
...,...,...,...,...,...,...,...
92,7732651739188276,20841,BOYDS,250533.236842,7.723518e+08,39.188277,-77.326518
93,7723271939208770,20876,GERMANTOWN,135940.688675,3.258196e+08,39.208771,-77.232719
94,7705685339203696,20833,BROOKEVILLE,207931.092330,5.585343e+08,39.203696,-77.056854
95,7743621739190007,20842,DICKERSON,357054.654503,1.131929e+09,39.190007,-77.436218
